In [ ]:
import pyprind
import pandas as pd
import os

labels = {'pos' : 1, 'neg' : 0}
pbar = pyprind.ProgBar(500000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join('aclImdb', s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 
                     'r', encoding = 'utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index = True)
            pbar.update()        

0% [##                            ] 100% | ETA: 00:15:40

In [9]:
df.columns = ['review', 'sentiment']

ValueError: Length mismatch: Expected axis has 0 elements, new values have 2 elements